### Colab 

In [ ]:
# Colab
'''
from google.colab import drive
drive.mount('/content/gdrive')
%cp -r gdrive/MyDrive/dance_generator/ .
%cd dance_generator
#!git pull
!pip install -r requirements.txt


'''

### Import and Config

In [1]:
import os
from model_setup import Model_Setup
from PoseDataGenerator import PoseDataGenerator
from Plot3D import save_image,save_gif,save_video
from Train import Train

# Folders
# Video folders to use for trainig
PATH_VIDEO = 'videos'
# CSV pose results of video folders
RESULT_CSV = 'results'
# produce the image for each N frames of csv file
IMAGE_PATH ='images'
# produce the image for each N frames of generate.csv
GENERATE_IMAGE_PATH ='generate_images'
# initial CSV pose as an input for generation
INIT_CSV = 'initial'
# The video that use for generation
PATH_VIDEO_GEN = 'generator'
# File of moddel that train has been done
MODEL_PATH = 'multihead_attention'

# Run parts
TRAIN = False
CREATE_IMAGES = False
CREATE_GIF = False
CREATE_VIDEO = False
POSE_GENERATION = False
CREATE_IMAGES_GEN = False
CREATE_VIDEO_GEN = False

# Model config
config = Model_Setup()
config.HIST_WINDOW = 10*24
config.MODEL_NAME = MODEL_PATH
config.HDF = "data.h5"
config.CREATE_HDF = True


### Generate training data from Video Clip

In [2]:
pose_data = PoseDataGenerator(config)
if POSE_GENERATION:
    if os.path.isfile(RESULT_CSV) == False:
        pose_data.generate_pose_multifile(PATH_VIDEO,RESULT_CSV)

### Testing 3DPlot Pose Estimation

In [ ]:
if CREATE_IMAGES:
    save_image(config,RESULT_CSV+'\coordinates_1.csv',IMAGE_PATH)

In [ ]:
if CREATE_GIF: 
    save_gif(IMAGE_PATH)
    
'''
from pygifsicle import optimize
gif_path = 'animated_from_images.gif'# create a new one
optimize(gif_path, 'animated_from_images_optimized.gif')# overwrite the original one
optimize(gif_path)
'''

In [ ]:
if CREATE_VIDEO: 
    save_video(IMAGE_PATH)

###  Train 


In [3]:
train_obj = Train(config)
if TRAIN:
    train_obj.fit(RESULT_CSV)
    train_obj.plot_performance()

### Generator

In [4]:
import pandas as pd

pose_data.generate_pose_multifile(PATH_VIDEO_GEN,INIT_CSV)
df_init = pd.read_csv(INIT_CSV+'/'+'coordinates.csv')

df_init = train_obj.dataset_df(df_init)
train_obj.generator(MODEL_PATH,df_init,frames_future=1000)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_4 (Bidirection (None, 240, 400)          480000    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 200)               400800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               25728     
_________________________________________________________________
dense_5 (Dense)              (None, 99)                12771     
Total params: 919,299
Trainable params: 919,299
Non-trainable params: 0
_________________________________________________________________


### Animation Generation

In [5]:
if CREATE_IMAGES_GEN:
    save_image(config,'generate.csv',GENERATE_IMAGE_PATH)
    #save_gif(GENERATE_IMAGE_PATH)

C:\Users\mhnaj\Desktop\dance_generator\Plot3D.py:159: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10, 10))


In [ ]:
if CREATE_VIDEO_GEN:
    save_video(GENERATE_IMAGE_PATH)
